In [150]:
from skimage import io
import matplotlib.pyplot as plt
import scanpy
import numpy as np
from scipy import ndimage
import os
import time
import pickle
import pandas as pd
from skimage import morphology
from scipy import ndimage as ndi
from scipy import stats

In [2]:
dataDir='/media/xinyi/T7 Shield/neuro'
phenoLabels=['Control','AD','FTLD-TDPC','PSP','IPD']
tif3dpath='TIFS/3DOrig'
segPath_3D='TIFS/segmentations_3D'
segPath_2D='TIFS/segmentations_new'
stains=np.array(['DAPI','GFAP','MAP2','LMNB'])
proj2dpath='TIFS/2DMaxProj'

In [3]:
savedir_processed=os.path.join(dataDir,'processed')
with open(os.path.join(savedir_processed,'allImgNames'), 'rb') as output:
    allImgNames=pickle.load(output)
with open(os.path.join(savedir_processed,'allCoord'), 'rb') as output:
    allCoord=pickle.load(output)

In [4]:
with open(os.path.join(savedir_processed,'allImgs_noMaxPerc_minmax_segNuc'), 'rb') as output:
    allImg=pickle.load(output)

In [5]:
with open(os.path.join(savedir_processed,'allPatientID'), 'rb') as output:
    allPatientID=pickle.load(output)
with open(os.path.join(savedir_processed,'allCat'), 'rb') as output:
    allCat=pickle.load(output)

#### invagination, plot examples

In [6]:
savedir_invag=os.path.join(savedir_processed,'invagination')
if not os.path.exists(savedir_invag):
    os.mkdir(savedir_invag)

In [139]:
def invagMask(img,structuring_element=morphology.disk(10),std_thresh=2):
    imgBinary=img>0
    imgBinary=ndi.binary_fill_holes(imgBinary)
    imgBinary_erode=morphology.binary_erosion(imgBinary,structuring_element)
    img_norim=np.copy(img)
    img_norim[np.logical_not(imgBinary_erode)]=0
    img_thresh=np.std(img_norim[img_norim>0])*std_thresh+np.median(img_norim[img_norim>0])
    img_invag=np.copy(img_norim)
    img_invag[img_invag<img_thresh]=0
    return img_invag

In [126]:
allImg.shape

(97530, 4, 128, 128)

In [127]:
invagMask(allImg[i,3]).shape

(128, 128)

In [140]:
allInvag=np.zeros((allImg.shape[0],allImg.shape[2],allImg.shape[3]))
for i in range(allImg.shape[0]):
    if i%500==0:
        print(i)
    allInvag[i]=invagMask(allImg[i,3])

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
8500

In [145]:
with open(os.path.join(savedir_invag,'allInvag'), 'wb') as output:
    pickle.dump(allInvag, output, pickle.HIGHEST_PROTOCOL)